# Async stream-validate LLM responses

Asynchronous behavior is generally useful in LLM applications. It allows multiple, long-running LLM requests to execute at once. 

With  streaming, you can make non-blocking, iterative validations over each stream as chunks arrive. This document explores how to implement this behavior using the Guardrails framework.

**Note**: learn more about streaming [here](./streaming).


In [ ]:
# Few imports and global variables
from rich import print
import guardrails as gd
from IPython.display import clear_output
import time

### Setup

Install the necessary validators from Guardrails hub in your CLI.

```bash
!guardrails hub install hub://guardrails/competitor_check
```


#### Create the Guard object


# Async Streaming

In [2]:
from guardrails.hub import CompetitorCheck

prompt = "Tell me about the Apple Iphone"

guard = gd.AsyncGuard().use(CompetitorCheck, ["Apple"])

##### Example 1: No async streaming

By default, the `stream` parameter is set to `False`.
We will use LiteLLM to make our LLM calls.


In [4]:
# Wrap the litellm OpenAI API call with the `guard` object
raw_llm_output, validated_output, *rest = await guard(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": prompt}],
    max_tokens=1024,
    temperature=0.3,
)

In [5]:
# Let's see the logs
print(guard.history.last.tree)

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭─────────────────────────────────────────────── Messages ────────────────────────────────────────────────╮ │
    │ │ ┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓                                                             │ │
    │ │ ┃   Role ┃ Content                        ┃                                                             │ │
    │ │ ┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩                                                             │ │
    │ │ │ system │ Tell me about the Apple Iphone │                                                             │ │
    │ │ └────────┴────────────────────────────────┘                                                             │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
    │ │ The iPhone is a line of smartphones designed and marketed by Apple Inc. It was first introduced in 2007 │ │
    │ │ by then-CEO Steve Jobs, revolutionizing the mobile phone industry with its innovative design and        │ │
    │ │ features. The iPhone runs on Apple's iOS operating system and is known for its sleek design,            │ │
    │ │ high-quality camera, and user-friendly interface.                                                       │ │
    │ │                                                                                                         │ │
    │ │ Over the years, Apple has released numerous models of the iPhone, each with upgraded features and       │ │
    │ │ capabilities. Some of the key features of the iPhone include Face ID facial recognition technology,     │ │
    │ │ Siri virtual assistant, and the App Store, which offers a wide range of apps for users to download.     │ │
    │ │                                                                                                         │ │
    │ │ The iPhone has become one of the most popular smartphones in the world, with a large and dedicated fan  │ │
    │ │ base. It is known for its high performance, reliability, and seamless integration with other Apple      │ │
    │ │ products and services. The iPhone continues to be a top-selling device, with new models being released  │ │
    │ │ regularly to keep up with the latest technological advancements.                                        │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
    │ │ The iPhone is a line of smartphones designed and marketed by Apple Inc. It was first introduced in 2007 │ │
    │ │ by then-CEO Steve Jobs, revolutionizing the mobile phone industry with its innovative design and        │ │
    │ │ features. The iPhone runs on Apple's iOS operating system and is known for its sleek design,            │ │
    │ │ high-quality camera, and user-friendly interface.                                                       │ │
    │ │                                                                                                         │ │
    │ │ Over the years, Apple has released numerous models of the iPhone, each with upgraded features and       │ │
    │ │ capabilities. Some of the key features of the iPhone include Face ID facial recognition technology,     │ │
    │ │ Siri virtual assistant, and the App Store, which offers a wide range of apps for users to download.     │ │
    │ │                                                                                                         │ │
    │ │ The iPhone has become one of the most popular smartphones in the world, with a large and dedicated fan  │ │
    │ │ base. It is known for its high performance,

##### Example 2: Async Streaming

Set the `stream` parameter to `True`


In [7]:
# Wrap the litellm OpenAI API call with the `guard` object
fragment_generator = await guard(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=1024,
    temperature=0,
    stream=True,
)


async for op in fragment_generator:
    clear_output(wait=True)
    print(op)
    time.sleep(0.5)

ValidationOutcome(
    call_id='14626972736',
    raw_llm_output='Overall, the iPhone is known for its user-friendly interface, high-quality build, and strong 
ecosystem of apps and services, making it a popular choice for consumers around the world.',
    validation_summaries=[],
    validated_output='\n\nOverall, the iPhone is known for its user-friendly interface, high-quality build, and 
strong ecosystem of apps and services, making it a popular choice for consumers around the world.',
    reask=None,
    validation_passed=True,
    error=None
)

In [9]:
# Let's see the logs
print(guard.history.last)

Call(
    id='14626972736',
    iterations=[
        Iteration(
            id='14626976016',
            index=0,
            call_id='14626972736',
            inputs=Inputs(
                llm_api=<guardrails.llm_providers.AsyncLiteLLMCallable object at 0x3677adbe0>,
                llm_output=None,
                instructions=None,
                prompt=None,
                messages=[{'role': 'user', 'content': Prompt(Tell me about the Apple Iphone)}],
                msg_history=None,
                prompt_params={},
                num_reasks=1,
                metadata={},
                full_schema_reask=False,
                stream=True
            ),
            outputs=Outputs(
                llm_response_info=LLMResponse(
                    prompt_token_count=None,
                    response_token_count=None,
                    output='',
                    stream_output=None,
                    async_stream_output=<openai.AsyncStream object at 0x3652e6c30>
                ),
                raw_output='',
                parsed_output='',
                validation_response='',
                guarded_output=None,
                reasks=[],
                validator_logs=[
                    ValidatorLogs(
                        validator_name='CompetitorCheck',
                        registered_name='guardrails/competitor_check',
                        instance_id=4403038784,
                        property_path='$',
                        value_before_validation='',
                        value_after_validation='',
                        validation_result=PassResult(
                            outcome='pass',
                            value_override=<class 
'guardrails.classes.validation.validation_result.PassResult.ValueOverrideSentinel'>,
                            metadata=None,
                            validated_chunk=None
                        ),
                        start_time=datetime.datetime(2024, 10, 11, 11, 15, 10, 595984),
                        end_time=datetime.datetime(2024, 10, 11, 11, 15, 10, 596421)
                    ),
                    ValidatorLogs(
                        validator_name='CompetitorCheck',
                        registered_name='guardrails/competitor_check',
                        instance_id=4403038784,
                        property_path='$',
                        value_before_validation='The',
                        value_after_validation='The',
                        validation_result=PassResult(
                            outcome='pass',
                            value_override=<class 
'guardrails.classes.validation.validation_result.PassResult.ValueOverrideSentinel'>,
                            metadata=None,
                            validated_chunk=None
                        ),
                        start_time=datetime.datetime(2024, 10, 11, 11, 15, 10, 596686),
                        end_time=datetime.datetime(2024, 10, 11, 11, 15, 10, 596991)
                    ),
                    ValidatorLogs(
                        validator_name='CompetitorCheck',
                        registered_name='guardrails/competitor_check',
                        instance_id=4403038784,
                        property_path='$',
                        value_before_validation=' Apple',
                        value_after_validation=' Apple',
                        validation_result=PassResult(
                            outcome='pass',
                            value_override=<class 
'guardrails.classes.validation.validation_result.PassResult.ValueOverrideSentinel'>,
                            metadata=None,
                            validated_chunk=None
                        ),
                        start_time=datetime.datetime(2024, 10, 11, 11, 15, 10, 614475),
                        end_time=datetime.datetime(2024, 10, 11, 11, 15, 10, 614755)
                    ),
                    ValidatorLogs(
       

As you can see here, the outputs in both examples match. The only difference is that, in the async streaming example, the outputs are returned as soon as they are received and validated by Guardrails.

In the non-streaming example, the outputs are returned only after the entire request has been processed by the API.

In other words, when async streaming is enabled, the API returns the outputs as soon as they are ready, rather than waiting for the entire request to be processed.
